# Skill estimation using Stan

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

import pickle

## Model that defined skill's distribution with more features

In [4]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> N1;             // Total number of old players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  real old_skill[N1];           //
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ 
    if (i <= N1) {skill[i] ~ normal(old_skill[i],1);}
    else {skill[i]~normal(0,3);} 
  }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [11]:
# try:     # load it if already compiled
#     sm = pickle.load(open('skill_model_new.pkl', 'rb'))
# except:  # ow, compile and save compiled model
#     sm = pystan.StanModel(model_code = skill_model)
#     with open('skill_model_new.pkl', 'wb') as f: pickle.dump(sm, f)

In [12]:
sm1 = pickle.load(open('skill_model.pkl', 'rb'))
sm2 = pickle.load(open('skill_model_new.pkl', 'rb'))

## Processing data

In [49]:
def load_data(dir='data/', new=0, pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins, wins_n = [], []
    playerA, playerB = [], []
    playerA_n, playerB_n = [], []
    nplayers = len(playerid) - new
    nplayers_n = new 
    nplays = np.zeros( (len(playerid),len(playerid)) )
    
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if a < nplayers and b < nplayers:
            if (np.random.rand() < pKeep):
                if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):

                    nplays[a,b] += 1
                    nplays[b,a]+=1                

                    playerA.append(a+1)
                    playerB.append(b+1)
                    wins.append(1 if aw else 0) 
        else:
            if (np.random.rand() < pKeep):
                if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                    nplays[a,b] += 1
                    nplays[b,a]+=1                      
                    
                    # involve new players
                    playerA_n.append(a+1)
                    playerB_n.append(b+1)
                    wins_n.append(1 if aw else 0) 

    return playerid, nplayers,playerA,playerB,wins,nplayers_n,playerA_n,playerB_n,wins_n

In [50]:
playerid, nplayers,playerA,playerB,wins,nplayers_n,playerA_n,playerB_n,wins_n = load_data(new=1)

In [51]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])

summary: 
# players 900
# games 2104
player A [468, 339, 160, 863, 50, 704, 84, 250, 669, 415]
player B [220, 479, 26, 310, 44, 239, 268, 249, 210, 40]
wins [0, 1, 0, 0, 1, 0, 1, 0, 0, 0]


In [52]:
print('summary: ')
print('# players', nplayers_n)
print('# games', len(wins_n))
print('player A', playerA_n[:10])
print('player B', playerB_n[:10])
print('wins', wins_n[:10])

summary: 
# players 99
# games 10931
player A [923, 976, 983, 954, 978, 945, 510, 389, 887, 538]
player B [155, 800, 228, 486, 313, 328, 947, 955, 998, 951]
wins [1, 1, 1, 0, 0, 0, 1, 0, 1, 0]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [53]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.3,
    'win':wins,
    'PA': playerA,
    'PB': playerB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

In [54]:
fit = sm1.sampling(data=skill_data, iter=1000, chains=4)

In [55]:
samples = fit.extract()

In [56]:
old_skills = samples['skill'].mean(0)
old_skills

array([ 5.70113698e-01,  2.78207137e+00,  5.36602495e-01,  3.48999487e+00,
        2.03588592e+00, -8.00916294e-01,  4.14680189e+00,  5.17175008e-01,
        3.63125588e+00,  9.76820936e-01,  1.03310221e+00,  4.23625625e+00,
        1.81586409e+00,  2.49307405e+00, -2.06210036e+00,  2.39938323e+00,
       -3.70312835e-01, -5.75825530e-01,  8.25664143e-02,  2.41003281e+00,
       -1.45214477e+00,  2.18410926e+00,  3.12058281e+00,  3.16169462e+00,
        9.15423047e-02,  2.54003156e+00,  2.52100392e+00, -1.24759035e+00,
        2.85772741e+00,  3.57598380e+00,  2.27373262e+00, -4.24177759e-01,
        1.80579728e+00,  3.40900097e+00,  1.20783829e+00, -2.05808257e+00,
       -1.20516553e-01,  2.33704687e+00,  8.98044320e-01,  1.65064891e+00,
        1.31033306e+00,  4.19287174e+00,  3.90164412e+00,  3.57632062e-02,
       -5.22662990e-01,  2.47881554e-01,  7.58890024e-01,  7.42520567e-02,
        5.29177169e-01,  3.42121991e+00,  1.42589760e+00,  3.71279005e+00,
        2.47202666e+00, -

new players

In [57]:
skill_data_new = {
    'N': nplayers+nplayers_n,
    'N1': nplayers,
    'E': len(wins_n),
    'scale': 0.3,
    'win':wins_n,
    'PA': playerA_n,
    'PB': playerB_n,
    'old_skill': old_skills
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [58]:
fit = sm2.sampling(data=skill_data_new, iter=1000, chains=4)

In [59]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [60]:
samples['skill'].shape # 2*100 iterations? 999 players

(2000, 999)

If we want to predict which player will win, we might use a direct estimator of that quantity based on the sample values:

In [62]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.3498901548841475


Remember to save the prediction!

In [63]:
with open('skill_hat_new.pkl', 'wb') as f: 
    pickle.dump(samples['skill'], f)

## Sample Model Evaluation

In [64]:
skill_hat = pickle.load(open('skill_hat_new.pkl', 'rb'))

In [71]:
def load_valid_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player  # id to name lookup


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
    
    return nplayers, nplays, nwins, games


In [72]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data()

In [73]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 188014.0
(999, 999) 94291.0
games 0


In [74]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [75]:
loss, binary_loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [76]:
loss, binary_loss

(0.4104668279686735, 0.44798030023204055)